### Imports und Client

In [29]:
import openrouteservice as ors
import folium

client = ors.Client(key='5b3ce3597851110001cf6248d2f185cd80644b67a7ca91371d7b6419')

# !!! ORS nutzt [lon, lat], folium nutzt [lat, lon] !!!
m = folium.Map(location=[52.521861, 13.40744], zoom_start=13)

### directions

In [ ]:
# …irgendwo in Berlin
coordinates = [[13.42731, 52.51088], [13.384116, 52.533558]]

route = client.directions(
    coordinates=coordinates,
    profile='foot-walking',
    format='geojson',
    options={"avoid_features": ["steps"]},
)

folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m) 

folium.Marker(location = list(reversed(coordinates[0])), icon=folium.Icon(color='green'), tooltip="Start").add_to(m)
folium.Marker(location = list(reversed(coordinates[1])), popup=folium.Popup("End")).add_to(m)

m

## Isochrones

In [ ]:
coordinate = [[13.384116, 52.533558]]

iso = client.isochrones(
    locations=coordinate,
    profile='foot-walking',
    range=[600, 1200],
    validate=False,
    attributes=['total_pop']
)

# Polygone werden in der angegebenen Reihenfolge übereinander gezeichnet, daher starten wir mit dem "größten"
for isochrone in reversed(iso['features']):
    folium.Polygon(locations=[list(reversed(coord)) for coord in isochrone['geometry']['coordinates'][0]],
                  fill='00ff00',
                  popup=folium.Popup("Population: {} people".format(isochrone['properties']['total_pop'])),
                  opacity=0.5).add_to(m)
    
m

## Matrix

In [ ]:
m = folium.Map(location=[52.521861, 13.40744], zoom_start=13)

coordinates = [[13.384116, 52.533558], [13.428726, 52.519355], [13.41774, 52.498929], [13.374825, 52.496369]]

matrix = client.distance_matrix(
    locations=coordinates,
    profile='foot-walking',
    metrics=['distance', 'duration'],
)

for marker in coordinates:
    folium.Marker(location=list(reversed(marker))).add_to(m)
    for other_marker in coordinates:
        if marker == other_marker:
            continue
        folium.PolyLine(
            locations = [list(reversed(marker)), list(reversed(other_marker))],
            tooltip=matrix['durations'][coordinates.index(marker)][coordinates.index(other_marker)]).add_to(m)

print("Durations in secs: {}\n".format(matrix['durations']))
print("Distances in m: {}".format(matrix['distances']))

m

### Forward Geocoding

In [ ]:
m = folium.Map(location=[52.521861, 13.40744], zoom_start=13)

address = "Mollstraße 1, Berlin"

geocode = client.pelias_search(
    text=address,
    focus_point=list(reversed(m.location)),
    validate=False,
)

for result in geocode['features']:
    folium.Marker(
        location=list(reversed(result['geometry']['coordinates'])),
        icon=folium.Icon(icon='compass', color='green', prefix='fa'),
        popup=folium.Popup(result['properties']['name'])
    ).add_to(m)

m

### Reverse Geocoding

In [ ]:
m = folium.Map(location=[52.521861, 13.40744], zoom_start=13)

coordinate = [13.413448, 52.490993]

reverse = client.pelias_reverse(
    point=coordinate,
    validate=False,
)

for result in reverse['features']:
    folium.Marker(
        location=list(reversed(result['geometry']['coordinates'])),
        icon=folium.Icon(icon='history', color='green', prefix='fa'),
        popup=folium.Popup(result['properties']['label'])
    ).add_to(m)

m